### 🏗️ Pipeline de Engenharia de Dados: Survey Visa/Embratur

**Objetivo:** Transformar dados brutos de pesquisa (formato *Wide*) em um Modelo Dimensional (Star Schema) otimizado para o Power BI.
**Entrada:** `codigo_dados.xlsx`
**Saída:** `d_respondente.csv`, `d_perguntas.csv`, `f_respostas.csv`

In [ ]:
# ==============================================================================
# 0. CONFIGURAÇÃO DO AMBIENTE (GOOGLE DRIVE) ☁️
# ==============================================================================
from google.colab import drive
import os

# 1. Monta o Google Drive
print("🔄 Conectando ao Google Drive...")
drive.mount('/content/drive')

# 2. Defina aqui o caminho da pasta onde você colocou o Excel no seu Drive
# DICA: Crie uma pasta chamada "Projeto_Visa" no seu Drive e jogue o excel lá
caminho_base = '/content/drive/My Drive/Projeto_Visa/'

# Verifica se a pasta existe para evitar erros depois
if os.path.exists(caminho_base):
    print(f"\n✅ Pasta encontrada com sucesso: {caminho_base}")
else:
    print(f"\n❌ A pasta '{caminho_base}' não foi encontrada.")
    print("   👉 Crie uma pasta 'Projeto_Visa' na raiz do seu Google Drive ou ajuste a variável 'caminho_base'.")

#### 1. Configuração e Extração (Extract)

**O que faz:** Carrega as bibliotecas, lê as abas do Excel e padroniza os nomes das colunas de metadados.
**Justificativa:** O Excel de origem possui nomes de colunas inconsistentes (ex: `descricao` em vez de `texto_pergunta`). Padronizar no início evita erros de *KeyError* nas etapas seguintes.

In [13]:
# Instalação de bibliotecas (Execute apenas uma vez se necessário)
# %pip install pandas openpyxl

import pandas as pd
import numpy as np
import warnings

# Configurações iniciais
warnings.filterwarnings('ignore') # Ignora avisos de versão para limpar a tela
pd.set_option('display.max_columns', None) # Garante que mostre todas as colunas se der print

# ==============================================================================
# 1. CONFIGURAÇÃO E CARGA (EXTRACT)
# ==============================================================================
print("🔄 1. Iniciando Pipeline: Carregando e Padronizando Dados...")

arquivo_excel = 'codigo_dados.xlsx'

try:
    # --- 1.1 Leitura das Abas do Excel ---
    print(f"   📂 Lendo arquivo: {arquivo_excel}...")
    df_data = pd.read_excel(arquivo_excel, sheet_name='data')
    df_variable = pd.read_excel(arquivo_excel, sheet_name='variable')
    df_datamap = pd.read_excel(arquivo_excel, sheet_name='datamap')
    print("   ✅ Arquivo carregado com sucesso.")

    # --- 1.2 Tratamento de Metadados (df_variable) ---
    # Renomeia coluna antiga 'descricao' se existir
    if 'descricao' in df_variable.columns:
        df_variable = df_variable.rename(columns={'descricao': 'texto_pergunta'})

    # Padroniza os nomes das colunas da aba 'variable'
    # Espera-se 3 colunas: Código da Variavel, Posição e Texto da Pergunta
    df_variable.columns = ['variavel', 'posicao', 'texto_pergunta']

    # --- 1.3 Tratamento do Mapa de Dados (df_datamap) ---
    # Padroniza os nomes das colunas da aba 'datamap'
    df_datamap.columns = ['variavel', 'codigo', 'label_resposta']

    # Higienização: Remove espaços em branco antes/depois do texto
    df_datamap['variavel'] = df_datamap['variavel'].astype(str).str.strip()
    df_datamap['label_resposta'] = df_datamap['label_resposta'].astype(str).str.strip()

    # Higienização: Garante que o código seja numérico inteiro
    df_datamap['codigo'] = pd.to_numeric(df_datamap['codigo'], errors='coerce')
    df_datamap = df_datamap.dropna(subset=['codigo']) # Remove linhas onde o código ficou vazio/erro
    df_datamap['codigo'] = df_datamap['codigo'].astype(int)

    print("   ✅ Metadados e Datamap higienizados e prontos.")

except FileNotFoundError:
    print(f"   ❌ ERRO CRÍTICO: O arquivo '{arquivo_excel}' não foi encontrado.")
    print("      Verifique se o nome do arquivo está correto e se ele está na mesma pasta.")
except Exception as e:
    print(f"   ❌ Erro inesperado ao carregar: {e}")

🔄 1. Iniciando Pipeline: Carregando e Padronizando Dados...
   📂 Lendo arquivo: codigo_dados.xlsx...
   ✅ Arquivo carregado com sucesso.
   ✅ Metadados e Datamap higienizados e prontos.


#### 2. Tratamento do Dicionário (Datamap)

Antes de traduzir os dados, precisamos garantir que o nosso "tradutor" (o Datamap) esteja limpo.

**Problema:** O Excel mistura números (`1`), textos (`"1"`) e floats (`1.0`), o que impede o Python de encontrar as chaves corretamente.

In [14]:
# ==============================================================================
# 2. SELEÇÃO DE DADOS (DIMENSÃO RESPONDENTE)
# ==============================================================================
print("🔄 2. Criando Dimensão Respondente (d_respondente)...")

# --- 2.1 Definição das Colunas de Interesse ---
# Organizado por categorias para facilitar a manutenção
colunas_perfil = [
    # Identificação e Demografia Básica
    'Respondent_Serial', 
    'GENDER_NonBinary', 'resp_age', 'QUOTAGERANGE',
    
    # Socioeconômico (Trabalho, Educação, Acompanhantes)
    'D1', 'D2', 
    'D31', 'D32', 'D33', 'D34', 'D35', 'D36',

    # Geografia (Residência)
    'S5_1_PAIS', 'S5_1_ESTADO', 

    # Características da Viagem
    'S3', 'S4', 'S6', 'S7', 'TIPO', 'TEMPORADA',

    # Metadados de Tempo (Usados para calcular a data)
    'CurrentDay', 'CurrentMonth', 'CurrentYear', 

    # Financeiro (Gastos e Renda)
    'GASTO_PESSOA',
    'Q24_1_VALOR', 'Q24_1_MOEDA', # Gasto Viagem
    'D4_1_VALOR', 'D4_1_MOEDA'    # Renda Familiar
]

# --- 2.2 Filtragem do Dataframe ---
# Seleciona apenas as colunas que realmente existem no Excel carregado
cols_existentes = [c for c in colunas_perfil if c in df_data.columns]
d_respondente = df_data[cols_existentes].copy()

# --- 2.3 Tratamento de Data e Ano (Onda) ---
# Verifica se as colunas de data existem antes de tentar processar
cols_data = ['CurrentYear', 'CurrentMonth', 'CurrentDay']
tem_colunas_data = all(col in d_respondente.columns for col in cols_data)

if tem_colunas_data:
    # Cria uma data única (YYYY-MM-DD)
    d_respondente['data_pesquisa'] = pd.to_datetime(
        d_respondente['CurrentYear'].astype(str) + '-' + 
        d_respondente['CurrentMonth'].astype(str) + '-' + 
        d_respondente['CurrentDay'].astype(str), 
        errors='coerce'
    )
    # Cria a coluna ONDA (Ano) para o filtro do Power BI
    d_respondente['Onda'] = d_respondente['CurrentYear'].fillna(2025).astype(int)
    
    # Remove as colunas originais para limpar
    d_respondente = d_respondente.drop(columns=cols_data, errors='ignore')
else:
    # Caso não tenha data, assume 2025 (Fallback de segurança)
    print("   ⚠️ Aviso: Colunas de data não encontradas. Assumindo data padrão.")
    d_respondente['Onda'] = 2025

print(f"   ✅ Tabela d_respondente criada com {d_respondente.shape[0]} linhas e {d_respondente.shape[1]} colunas.")

🔄 2. Criando Dimensão Respondente (d_respondente)...
   ✅ Tabela d_respondente criada com 8131 linhas e 27 colunas.


#### 3. Construção da Dimensão Respondente (`d_respondente`)

Esta tabela contém o **perfil único** de cada turista.

**Transformações:**

* **Data:** Unificação de colunas separadas (Dia/Mês/Ano).
* **Tradução Robusta:** Conversão de códigos (`1`, `2`) para texto (`Sim`, `Não`) usando lógica híbrida (Excel + Dicionários Manuais para corrigir falhas na origem).
* **Renomeação:** Padronização para *snake_case* (ex: `id_respondente`).

In [15]:
# ==============================================================================
# 3. TRADUÇÃO E NORMALIZAÇÃO FINANCEIRA (TRANSFORM)
# ==============================================================================
print("🔄 3. Traduzindo Códigos e Convertendo Moedas...")

# --- 3.1 Dicionários Manuais e Taxas (Setup) ---
# Usados quando o dicionário do Excel falha ou para agrupar respostas
dic_d1 = {
    1: 'Empregado em tempo integral', 2: 'Empregado em tempo parcial', 3: 'Autônomo',
    4: 'Desempregado, mas procurando emprego', 5: 'Desempregado e não procurando emprego/Doente ou incapacitado a longo prazo', 
    6: 'Cuidador(a) de tempo integral/Dono(a) de casa', 7: 'Aposentado(a)', 8: 'Apenas estudante', 9: 'Empregado em regime remoto'
}

dic_d2 = {
    1: 'Sem estudos', 2: 'Fundamental incompleto', 3: 'Fundamental completo',
    4: 'Médio incompleto', 5: 'Médio completo', 6: 'Superior incompleto',
    7: 'Superior completo', 8: 'Pós-Graduação incompleto/completo', 
    9: 'Mestrado incompleto/completo', 10: 'Doutorado incompleto/completo'
}

dic_d3x = {
    1: 'Apenas eu', 2: 'Meu companheiro(a)', 3: 'Meu(s) filho(s)',
    4: 'Meu pai e/ou mãe', 5: 'Irmãos ou outros parentes', 6: 'Amigos'
}

dic_moedas = {
    1: 'ARS (Peso argentino)', 2: 'USD (Dólar americano)', 3: 'CLP (Peso chileno)', 
    4: 'PYG (Guarani)', 5: 'UYU (Peso Uruguaio)', 6: 'EUR (EURO)', 
    7: 'GBP (Libra esterlina)', 8: 'CAD (Dólar canadense)', 9: 'COP (Peso colombiano)', 
    10: 'PEN (Sol)', 11: 'MXN (Peso mexicano)', 12: 'BRL (Real)'
}

# Taxas estimadas para conversão (Base 1 USD)
taxas_cambio = {
    'ARS': 0.00069, 'USD': 1.0, 'CLP': 0.0011, 'PYG': 0.00015,
    'UYU': 0.0256, 'EUR': 1.18, 'GBP': 1.35, 'CAD': 0.73,
    'COP': 0.00026, 'PEN': 0.29, 'MXN': 0.056, 'BRL': 0.18
}

# --- 3.2 Funções Auxiliares ---

def traduzir_valor_robusto(valor, dicionario):
    """Traduz códigos (1, 2) para texto, lidando com múltiplas respostas (1;3)."""
    if pd.isna(valor) or str(valor).strip() == '': return valor
    texto_valor = str(valor)
    partes = texto_valor.split(';') 
    partes_traduzidas = []
    for parte in partes:
        codigo_limpo = parte.strip().split('.')[0]
        try:
            cod_int = int(codigo_limpo)
            traducao = dicionario.get(cod_int, parte)
            partes_traduzidas.append(str(traducao))
        except ValueError:
            partes_traduzidas.append(parte)
    return "; ".join(partes_traduzidas)

def converter_para_usd(row, col_valor, col_moeda):
    """Converte valor para USD baseado na coluna de moeda correspondente."""
    try:
        valor = float(row[col_valor])
        moeda_texto = str(row[col_moeda]).upper()
        
        # Tenta achar a taxa pela sigla (ex: BRL) ou palavras-chave
        taxa = 1.0
        encontrou = False
        
        for moeda, valor_taxa in taxas_cambio.items():
            if moeda in moeda_texto:
                taxa = valor_taxa
                encontrou = True
                break
        
        if not encontrou:
            if 'REAL' in moeda_texto: taxa = 0.20
            elif 'EURO' in moeda_texto: taxa = 1.08
            elif 'DOLAR' in moeda_texto or 'DÓLAR' in moeda_texto: taxa = 1.0
            elif 'PESO ARGENTINO' in moeda_texto: taxa = 0.0012

        return round(valor * taxa, 2)
    except:
        return np.nan

# --- 3.3 Execução: Tradução de Colunas ---
cols_para_traduzir = [
    'GENDER_NonBinary', 'TEMPORADA', 'TIPO', 'S5_1_PAIS', 'S3', 'S4', 'S6', 'S7', 'QUOTAGERANGE',
    'D1', 'D2', 'D31', 'D32', 'D33', 'D34', 'D35', 'D36',
    'Q24_1_MOEDA', 'D4_1_MOEDA' 
]

for col in cols_para_traduzir:
    if col in d_respondente.columns:
        dic_atual = {}
        
        # Seleciona o dicionário correto
        if col == 'D1': dic_atual = dic_d1
        elif col == 'D2': dic_atual = dic_d2
        elif col.startswith('D3'): dic_atual = dic_d3x
        elif 'MOEDA' in col: dic_atual = dic_moedas
        else:
            # Busca no Datamap do Excel
            mapa_var = df_datamap[df_datamap['variavel'] == col]
            if not mapa_var.empty:
                dic_atual = dict(zip(mapa_var['codigo'], mapa_var['label_resposta']))
        
        # Aplica a tradução se encontrou dicionário
        if dic_atual:
            d_respondente[col] = d_respondente[col].apply(lambda x: traduzir_valor_robusto(x, dic_atual))

# --- 3.4 Execução: Conversão Financeira ---
if 'Q24_1_VALOR' in d_respondente.columns and 'Q24_1_MOEDA' in d_respondente.columns:
    print("   Calculando Gasto Viagem (USD)...")
    d_respondente['gasto_viagem_usd'] = d_respondente.apply(lambda x: converter_para_usd(x, 'Q24_1_VALOR', 'Q24_1_MOEDA'), axis=1)

if 'D4_1_VALOR' in d_respondente.columns and 'D4_1_MOEDA' in d_respondente.columns:
    print("   Calculando Renda Familiar (USD)...")
    d_respondente['renda_familiar_usd'] = d_respondente.apply(lambda x: converter_para_usd(x, 'D4_1_VALOR', 'D4_1_MOEDA'), axis=1)

# --- 3.5 Renomeação Final ---
renomear_cols = {
    'Respondent_Serial': 'id_respondente', 
    'GENDER_NonBinary': 'genero', 
    'resp_age': 'idade',
    'QUOTAGERANGE': 'faixa_etaria', 
    'S5_1_PAIS': 'pais', 
    'S5_1_ESTADO': 'estado_residencia', 
    'S3': 'reside_brasil', 
    'S4': 'viagem_int_12m', 
    'S6': 'meio_transporte', 
    'S7': 'portao_entrada',
    'TIPO': 'tipo_turista', 
    'TEMPORADA': 'temporada', 
    'GASTO_PESSOA': 'gasto_total_calculado',
    'D1': 'status_emprego', 
    'D2': 'escolaridade',
    'D31': 'd31', 'D32': 'd32', 'D33': 'd33', 'D34': 'd34', 'D35': 'd35', 'D36': 'd36',
    'Q24_1_VALOR': 'gasto_viagem_original', 
    'Q24_1_MOEDA': 'moeda_viagem',
    'D4_1_VALOR': 'renda_familiar_original', 
    'D4_1_MOEDA': 'moeda_renda'
}
d_respondente = d_respondente.rename(columns=renomear_cols)

# Ajuste estético na coluna de estado de residência
if 'estado_residencia' in d_respondente.columns:
    d_respondente['estado_residencia'] = d_respondente['estado_residencia'].astype(str).str.title().str.strip().replace({'Nan': 'Não Informado', 'Null': 'Não Informado'})

print("   ✅ d_respondente processada com sucesso.")

🔄 3. Traduzindo Códigos e Convertendo Moedas...
   Calculando Gasto Viagem (USD)...
   Calculando Renda Familiar (USD)...
   ✅ d_respondente processada com sucesso.


#### 4. Construção da Fato Respostas (`f_respostas`)

Esta é a tabela longa (Unpivoted) que contém todas as respostas. 

**Otimizações:**

* **Melt:** Transforma colunas em linhas.
* **Limpeza de Nulos:** Remove linhas vazias para reduzir drasticamente o tamanho do arquivo.
* **Fallback de Texto:** Se não houver tradução (ex: "Outro país: Chile"), mantém o texto original digitado pelo usuário.

In [16]:
# ==============================================================================
# 4. CRIAÇÃO DA FATO RESPOSTAS (f_respostas)
# ==============================================================================
print("🔄 4. Criando Fato Respostas (f_respostas)...")

# --- 4.1 Preparação e Melt (Transformar Colunas em Linhas) ---
# Recarrega dados originais para garantir integridade
df_data_raw = pd.read_excel(arquivo_excel, sheet_name='data')

# Lista de colunas a IGNORAR na Fato (porque já estão na Dimensão Respondente ou são financeiras)
cols_ignorar = [
    'Respondent_Serial', 'CurrentDay', 'CurrentMonth', 'CurrentYear', 
    'GENDER_NonBinary', 'resp_age', 'QUOTAGERANGE', 'S5_1_PAIS', 'S5_1_ESTADO', 'TIPO',
    'Q24_1_VALOR', 'Q24_1_MOEDA', 'D4_1_VALOR', 'D4_1_MOEDA' 
]
cols_fato = [c for c in df_data_raw.columns if c not in cols_ignorar]

# Unpivot
f_respostas = df_data_raw.melt(
    id_vars=['Respondent_Serial'], 
    value_vars=cols_fato, 
    var_name='cod_pergunta', 
    value_name='cod_resposta'
)

# Limpeza Inicial (Remove Vazios)
f_respostas = f_respostas.dropna(subset=['cod_resposta'])
f_respostas = f_respostas[f_respostas['cod_resposta'] != '']

# --- 4.2 Tradução via Datamap (Padrão) ---
f_respostas = f_respostas.merge(
    df_datamap, 
    left_on=['cod_pergunta', 'cod_resposta'], 
    right_on=['variavel', 'codigo'], 
    how='left'
)

# Se não achou tradução no Datamap, usa o próprio código (Fallback)
f_respostas['resposta_texto'] = f_respostas['label_resposta'].fillna(f_respostas['cod_resposta'])

# ==============================================================================
# 🔧 CORREÇÕES ESPECÍFICAS E TRATAMENTO DE DADOS
# ==============================================================================
print("   🔧 Aplicando correções manuais (Gastronomia, Acomodação, Atividades e Mapa)...")

# --- 4.3 Definição dos Dicionários de Correção ---

dic_acomodacao_fix = {
    1: 'Hotel de 3 estrelas ou menos', 2: 'Hotel de 4 ou 5 estrelas', 3: 'Pousada',
    4: 'Resort', 5: 'Casa de amigos e parentes', 6: 'Imóvel próprio',
    7: 'Imóvel alugado por temporada', 8: 'Airbnb e similares', 9: 'Albergue/hostel',
    10: 'Camping', 11: 'Cruzeiro / Navio', 98: 'Outros'
}

dic_gastronomia_fix = {
    1: 'Restaurante de alta gastronomia', 2: 'Restaurante por quilo/buffet', 3: 'Fast Food',
    4: 'Cafeterias', 5: 'Étnicos (de alguma região específica)', 6: 'Bistrô',
    7: 'Food truck', 8: 'Restaurantes à la carte', 98: 'Outros'
}

# --- Dicionário de Atividades (Q23) ---
dic_atividades_fix = {
    1: 'Tratamentos médicos/hospitalares ou estéticos', 2: 'Terapias de bem-estar',
    4: 'Turismo místico/esotérico', 5: 'Atividades náuticas (barco, vela)',
    6: 'Cruzeiros marítimos ou fluviais', 7: 'Visitas culturais (museus, patrimônio histórico)',
    8: 'Visitas a comunidades tradicionais (indígena, quilombola)',
    9: 'Manifestações/eventos culturais (shows, festas típicas)',
    10: 'Gastronomia (alta cozinha/típica)', 11: 'Prática de esportes (exceto aventura)',
    12: 'Assistir a eventos esportivos', 13: 'Mergulho',
    14: 'Atividades de aventura (trilhas, rafting, buggy)', 15: 'Observação de fauna e flora',
    16: 'Turismo rural / campo', 17: 'Visita a parques temáticos',
    18: 'Ecoturismo (cachoeiras, parques naturais)', 19: 'Compras pessoais',
    20: 'Visitar amigos e parentes', 21: 'Vida Noturna e entretenimento',
    22: 'Sol e praia', 23: 'Participar do carnaval de rua',
    24: 'Assistir carnaval no sambódromo', 98: 'Outros', 99: 'Não realizou atividades'
}

# --- 4.4 Função de Patch (Correção por Padrão de Pergunta) ---
def aplicar_patch_padrao(df, padrao_regex, dicionario):
    mask = df['cod_pergunta'].astype(str).str.contains(padrao_regex, regex=True, na=False)
    if mask.sum() > 0:
        def traduzir_item(valor):
            if pd.isna(valor) or str(valor).strip() == '': return valor
            valor_str = str(valor).strip()
            try:
                cod_limpo = int(float(valor_str))
                return dicionario.get(cod_limpo, valor_str)
            except:
                partes = valor_str.split(';')
                partes_traduzidas = [str(dicionario.get(int(float(p)), p)) if p.replace('.','',1).isdigit() else p for p in partes]
                return "; ".join(partes_traduzidas)
        df.loc[mask, 'resposta_texto'] = df.loc[mask, 'cod_resposta'].apply(traduzir_item)
    return df

# Aplica as correções
f_respostas = aplicar_patch_padrao(f_respostas, r'^Q15|^Q16', dic_acomodacao_fix)
f_respostas = aplicar_patch_padrao(f_respostas, r'^Q18', dic_gastronomia_fix)
f_respostas = aplicar_patch_padrao(f_respostas, r'^Q23', dic_atividades_fix)

# --- 4.5 LIMPEZA GERAL DE "OUTROS" (NOVO) --- 🚨 AQUI ESTÁ A CORREÇÃO
# Substitui qualquer variação de "Outros. Quais?" por apenas "Outros"
f_respostas['resposta_texto'] = f_respostas['resposta_texto'].astype(str).replace({
    'Outros. Quais?': 'Outros',
    'Outros. Quais': 'Outros',
    'Outros (especificar)': 'Outros'
})

# --- 4.6 GERAÇÃO DAS COLUNAS PARA O MAPA DO POWER BI ---
f_respostas['UF_Mapa'] = f_respostas['resposta_texto'].astype(str).str.extract(r'\((.*?)\)')
dic_mapa_bi = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapa', 'AM': 'Amazonas', 'BA': 'Bahia', 'CE': 'Ceara',
    'DF': 'Distrito Federal', 'ES': 'Espirito Santo', 'GO': 'Goias', 'MA': 'Maranhao', 'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul', 'MG': 'Minas Gerais', 'PA': 'Para', 'PB': 'Paraiba', 'PR': 'Parana',
    'PE': 'Pernambuco', 'PI': 'Piaui', 'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul', 'RO': 'Rondonia', 'RR': 'Roraima', 'SC': 'Santa Catarina',
    'SP': 'Sao Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins'
}
f_respostas['Nome_Estado_Mapa'] = f_respostas['UF_Mapa'].map(dic_mapa_bi)

# --- 4.7 Finalização ---
f_respostas = f_respostas.rename(columns={'Respondent_Serial': 'id_respondente'})
f_respostas = f_respostas[['id_respondente', 'cod_pergunta', 'cod_resposta', 'resposta_texto', 'Nome_Estado_Mapa']]
f_respostas['resposta_texto'] = f_respostas['resposta_texto'].astype(str).str.strip()

print(f"   ✅ f_respostas criada com {len(f_respostas)} linhas.")
print("   ✅ Limpeza de 'Outros. Quais?' aplicada globalmente.")

🔄 4. Criando Fato Respostas (f_respostas)...
   🔧 Aplicando correções manuais (Gastronomia, Acomodação, Atividades e Mapa)...
   ✅ f_respostas criada com 1443985 linhas.
   ✅ Limpeza de 'Outros. Quais?' aplicada globalmente.


#### 5. Construção da Dimensão Perguntas (`d_perguntas`)

Tabela de metadados para criar menus e filtros no Dashboard.

**Transformações:**

* **Categorização:** Criação da coluna `categoria` baseada nos prefixos (Q1, Q24, S...) para permitir navegação por menu.
* **Limpeza:** Remoção de tags técnicas (`[HIDDEN]`) e caracteres indesejados.

In [17]:
# ==============================================================================
# 5. CRIAÇÃO DA DIMENSÃO PERGUNTAS (d_perguntas)
# ==============================================================================
print("🔄 5. Criando Dimensão Perguntas (d_perguntas)...")

# --- 5.1 Seleção Inicial ---
d_perguntas = df_variable[['variavel', 'texto_pergunta']].copy()
d_perguntas.columns = ['cod_pergunta', 'texto_pergunta']

# --- 5.2 Lógica de Categorização (Menu de Navegação) ---
def definir_categoria(codigo):
    codigo = str(codigo).upper().strip()
    
    # 1. Perfil
    if (codigo.startswith('D') and len(codigo) > 1 and codigo[1].isdigit()) or \
       codigo in ['GENDER_NONBINARY', 'RESP_AGE', 'QUOTAGERANGE', 'S5_1_PAIS', 'S5_1_ESTADO', 'Q1', 'Q2']:
        return '1. Perfil do Turista'
    
    # 2. Planejamento
    if codigo.startswith(('Q3', 'Q4', 'Q5', 'Q6')) or 'Q23A' in codigo:
        return '2. Planejamento e Marketing'
    
    # 3. Características da Viagem
    if codigo.startswith(('S', 'Q7', 'Q8', 'Q9', 'Q10', 'Q14', 'Q19', 'Q20', 'Q21', 'Q22')):
        return '3. Características da Viagem'
    
    # 4. Destinos
    if codigo.startswith(('Q11', 'Q12', 'Q13')):
        return '4. Destinos Visitados'
    
    # 5. Logística
    if codigo.startswith(('Q15', 'Q16', 'Q17')):
        return '5. Hospedagem e Transporte'
    
    # 6. Atividades
    if codigo.startswith(('Q18', 'Q23')): 
        return '6. Atividades e Gastronomia'
    
    # 7. Financeiro
    if codigo.startswith(('Q24', 'Q25', 'Q26', 'Q27', 'Q28', 'Q29', 'Q30', 'Q31', 'Q32')) or 'GASTO' in codigo:
        return '7. Gastos e Pagamentos'
    
    # 8. Avaliação
    if codigo.startswith(('Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38', 'Q39')):
        return '8. Avaliação e Imagem'

    return '9. Outros / Técnico'

# Aplica a categorização
d_perguntas['categoria'] = d_perguntas['cod_pergunta'].apply(definir_categoria)

# --- 5.3 Filtragem de "Lixo Técnico" ---
# Identifica quais perguntas realmente têm dados na tabela Fato
perguntas_na_fato = f_respostas['cod_pergunta'].unique()

# Lista de colunas técnicas para remover
lixo_tecnico = ['TIPO', 'RESPONDENT_SERIAL', 'CURRENTDAY', 'CURRENTMONTH', 'CURRENTYEAR']

# Cria a máscara de exclusão
filtro_lixo = d_perguntas['cod_pergunta'].str.upper().isin(lixo_tecnico) | \
              d_perguntas['cod_pergunta'].str.contains('_Coded_', na=False) | \
              d_perguntas['cod_pergunta'].str.contains('_SEM_OUTLIERS', na=False) | \
              ~d_perguntas['cod_pergunta'].isin(perguntas_na_fato) # Remove se não estiver na Fato

d_perguntas = d_perguntas[~filtro_lixo].copy()

# --- 5.4 Limpeza Visual dos Textos ---
d_perguntas['texto_pergunta'] = d_perguntas['texto_pergunta'].astype(str).str.strip()
# Remove dois pontos no final (ex: "Idade:")
d_perguntas['texto_pergunta'] = d_perguntas['texto_pergunta'].str.replace(r'\s*:\s*$', '', regex=True)
# Remove tags entre colchetes (ex: "País [Única]")
d_perguntas['texto_pergunta'] = d_perguntas['texto_pergunta'].str.replace(r'\[.*?\]', '', regex=True)

print(f"   ✅ d_perguntas criada com {len(d_perguntas)} perguntas categorizadas.")

🔄 5. Criando Dimensão Perguntas (d_perguntas)...
   ✅ d_perguntas criada com 409 perguntas categorizadas.


#### 6. Exportação (Load)

Salva os Dataframes em CSV prontos para importação no Power BI. Usamos `;` como separador para evitar conflitos com vírgulas nos textos das perguntas.

In [18]:
# ==============================================================================
# 6. HIGIENIZAÇÃO FINAL E EXPORTAÇÃO (LOAD)
# ==============================================================================
print("🔄 6. Iniciando Higienização Final e Exportação...")

# --- 6.1 Função de Limpeza de Texto ---
def limpar_quebras(df, nome_tabela):
    """Remove quebras de linha (\n) que quebram a importação no Power BI."""
    print(f"   🧹 Limpando tabela: {nome_tabela}...")
    df_limpo = df.copy()
    
    # Itera sobre colunas de texto
    cols_texto = df_limpo.select_dtypes(include=['object', 'string']).columns
    
    for col in cols_texto:
        df_limpo[col] = df_limpo[col].astype(str)
        # Remove Enter (\n) e Retorno de Carro (\r)
        df_limpo[col] = df_limpo[col].str.replace(r'[\r\n]+', ' ', regex=True)
        # Remove espaços extras nas pontas
        df_limpo[col] = df_limpo[col].str.strip()
        # Remove textos que representam vazio
        df_limpo[col] = df_limpo[col].replace({'nan': '', 'None': '', 'NaT': ''})
        
    return df_limpo

# Aplica a limpeza
d_respondente = limpar_quebras(d_respondente, "d_respondente")
d_perguntas = limpar_quebras(d_perguntas, "d_perguntas")
f_respostas = limpar_quebras(f_respostas, "f_respostas")

# --- 6.2 Exportação para CSV ---
print("   💾 Salvando arquivos CSV...")

try:
    # Encoding 'utf-8-sig' é essencial para acentos no Excel/Power BI
    # Separador ';' é melhor para o padrão brasileiro
    d_perguntas.to_csv('d_perguntas.csv', index=False, encoding='utf-8-sig', sep=';')
    d_respondente.to_csv('d_respondente.csv', index=False, encoding='utf-8-sig', sep=';')
    f_respostas.to_csv('f_respostas.csv', index=False, encoding='utf-8-sig', sep=';')
    
    print("-" * 50)
    print("🚀 SUCESSO TOTAL! Pipeline concluído.")
    print("   Arquivos gerados na pasta:")
    print("   1. d_perguntas.csv")
    print("   2. d_respondente.csv")
    print("   3. f_respostas.csv")
    print("-" * 50)

except PermissionError:
    print("\n❌ ERRO DE PERMISSÃO!")
    print("   Parece que um dos arquivos CSV está aberto no Excel ou Power BI.")
    print("   👉 Feche os arquivos e rode esta célula novamente.")
except Exception as e:
    print(f"\n❌ Erro inesperado ao salvar: {e}")

🔄 6. Iniciando Higienização Final e Exportação...
   🧹 Limpando tabela: d_respondente...
   🧹 Limpando tabela: d_perguntas...
   🧹 Limpando tabela: f_respostas...
   💾 Salvando arquivos CSV...
--------------------------------------------------
🚀 SUCESSO TOTAL! Pipeline concluído.
   Arquivos gerados na pasta:
   1. d_perguntas.csv
   2. d_respondente.csv
   3. f_respostas.csv
--------------------------------------------------
